In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [6]:
# Require cuda and cudnn
assert torch.cuda.is_available()
assert torch.backends.cudnn.enabled

In [ ]:
%run ../../lib/download.py